In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
from tensorboardX import SummaryWriter
from torch import optim
from torch.optim.lr_scheduler import StepLR

import os
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import copy
from tqdm.notebook import tqdm
import torch.backends.cudnn as cudnn

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'                # GPU Number 
start_time = time.time()
batch_size = 128
learning_rate = 0.008
default_directory = './save_models'
writer = SummaryWriter('./log/vgg19_prelu_dropblock_cosine_1') #!#

In [3]:
transformer_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),               # Random Position Crop
    transforms.RandomHorizontalFlip(),                  # right and left flip
    #transforms.ColorJitter(brightness=(0.2, 2), contrast=(0.3, 2), saturation=(0.2, 2), hue=(-0.3, 0.3)),
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize(mean=(0.4914, 0.4824, 0.4467), # RGB Normalize MEAN
                         std=(0.2471, 0.2436, 0.2616))  # RGB Normalize Standard Deviation
])

transform_test = transforms.Compose([
    transforms.ToTensor(),                              # change [0,255] Int value to [0,1] Float value
    transforms.Normalize(mean=(0.4914, 0.4824, 0.4467), # RGB Normalize MEAN
                         std=(0.2471, 0.2436, 0.2616))  # RGB Normalize Standard Deviation
])

training_dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transformer_train)
#training_dataset_2 = datasets.CIFAR10('./data', train=True, download=True, transform=transformer_train)
#training_dataset_3 = datasets.CIFAR10('./data', train=True, download=True, transform=transformer_train)
validation_dataset = datasets.CIFAR10('./data', train=False, download=True, transform=transform_test)

training_loader = torch.utils.data.DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
#training_loader_2 = torch.utils.data.DataLoader(dataset=training_dataset_2, batch_size=batch_size, shuffle=True)
#training_loader_3 = torch.utils.data.DataLoader(dataset=training_dataset_3, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
class DropBlock2D(nn.Module):
 
    def __init__(self, drop_prob, block_size):
        super(DropBlock2D, self).__init__()

        self.drop_prob = drop_prob
        self.block_size = block_size

    def forward(self, x):
        # shape: (bsize, channels, height, width)

        assert x.dim() == 4, \
            "Expected input with 4 dimensions (bsize, channels, height, width)"

        if not self.training or self.drop_prob == 0.:
            return x
        else:
            # get gamma value
            gamma = self._compute_gamma(x)

            # sample mask
            mask = (torch.rand(x.shape[0], *x.shape[2:]) < gamma).float()

            # place mask on input device
            mask = mask.to(x.device)

            # compute block mask
            block_mask = self._compute_block_mask(mask)

            # apply block mask
            out = x * block_mask[:, None, :, :]

            # scale output
            out = out * block_mask.numel() / block_mask.sum()

            return out

    def _compute_block_mask(self, mask):
        block_mask = F.max_pool2d(input=mask[:, None, :, :],
                                  kernel_size=(self.block_size, self.block_size),
                                  stride=(1, 1),
                                  padding=self.block_size // 2)

        if self.block_size % 2 == 0:
            block_mask = block_mask[:, :, :-1, :-1]

        block_mask = 1 - block_mask.squeeze(1)

        return block_mask

    def _compute_gamma(self, x):
        return self.drop_prob / (self.block_size ** 2)

In [5]:
class LinearScheduler(nn.Module):
    def __init__(self, dropblock, start_value, stop_value, nr_steps):
        super(LinearScheduler, self).__init__()
        self.dropblock = dropblock
        self.i = 0
        self.drop_values = np.linspace(start=start_value, stop=stop_value, num=int(nr_steps))

    def forward(self, x):
        return self.dropblock(x)

    def step(self):
        if self.i < len(self.drop_values):
            self.dropblock.drop_prob = self.drop_values[self.i]

        self.i += 1

In [6]:
vgg_types = {
                'VGG11' : [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
                'VGG13' : [64, 64 ,'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
                'VGG16' : [64, 64 ,'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
                'VGG19' : [64, 64 ,'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
                
                }

class VGGnet(nn.Module):
    def __init__(self, model, in_channels=3, num_classes=10, init_weights=True):
        super(VGGnet, self).__init__()
        self.in_channels = in_channels

        #self.conv_layers = self.create_conv_laters(vgg_types[model])

        self.conv_layers_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.PReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

        self.conv_layers_2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.PReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.PReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

        self.conv_layers_3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.PReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.PReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.PReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.PReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

        self.conv_layers_4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.ELU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

        self.conv_layers_5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(512),
            nn.PReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        )

        self.dropblock = LinearScheduler(DropBlock2D(drop_prob=0.35, block_size=4), start_value=0.35, stop_value=0.55, nr_steps=60000)
           
        
            

        self.fcs = nn.Sequential(
            nn.Linear(512, 4096),
            nn.PReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.PReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        self.dropblock.step()
        x = self.conv_layers_1(x)
        x = self.conv_layers_2(x)
        x = self.dropblock(x)
        x = self.conv_layers_3(x)
        x = self.conv_layers_4(x)
        x = self.dropblock(x)
        x = self.conv_layers_5(x)
        x = x.view(x.size(0), -1)
        x = self.fcs(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


# creat VGGnet object
model = VGGnet('VGG19', in_channels=3, num_classes=10, init_weights=True)

In [7]:
if torch.cuda.device_count() > 0:
    print("USE", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model).cuda()
    cudnn.benchmark = True
else:
    print("USE ONLY CPU!")

summary(model, (3, 32,32))

USE 1 GPUs!
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
             PReLU-3           [-1, 64, 32, 32]               1
            Conv2d-4           [-1, 64, 32, 32]          36,928
       BatchNorm2d-5           [-1, 64, 32, 32]             128
             PReLU-6           [-1, 64, 32, 32]               1
         MaxPool2d-7           [-1, 64, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]          73,856
       BatchNorm2d-9          [-1, 128, 16, 16]             256
            PReLU-10          [-1, 128, 16, 16]               1
           Conv2d-11          [-1, 128, 16, 16]         147,584
      BatchNorm2d-12          [-1, 128, 16, 16]             256
            PReLU-13          [-1, 128, 16, 16]               1
        MaxPool2d-14       

In [8]:
optimizer = optim.SGD(model.parameters(), learning_rate,
                                momentum=0.9,
                                weight_decay=1e-4,
                                nesterov=True)             
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=3, eta_min=0.001)
criterion = nn.CrossEntropyLoss()

In [9]:
def train(epoch):
    model.train()
    train_loss = 0 
    total = 0
    correct = 0
    iters = len(training_loader)
    for batch_idx, (data, target) in enumerate(training_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        scheduler.step(epoch + batch_idx / iters)
        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)

        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()
        if batch_idx % 10 == 0:
            print('Epoch: {} | Batch_idx: {} |  Loss_1: ({:.4f}) | Acc_1: ({:.2f}%) ({}/{})'
                  .format(epoch, batch_idx, train_loss / (batch_idx + 1), 100. * correct / total, correct, total))

        writer.add_scalar('training loss', (train_loss / (batch_idx + 1)) , epoch * len(training_loader) + batch_idx) #!#
        writer.add_scalar('training accuracy', (100. * correct / total), epoch * len(training_loader) + batch_idx) #!#
        writer.add_scalar('lr', optimizer.param_groups[0]['lr'], epoch * len(training_loader) + batch_idx) #!#

In [10]:
def test(epoch):
    model.eval()

    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(validation_loader):
        if torch.cuda.is_available():
            data, target = Variable(data.cuda()), Variable(target.cuda())
        else:
            data, target = Variable(data), Variable(target)

        outputs = model(data)
        loss = criterion(outputs, target)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += predicted.eq(target.data).cpu().sum()

        writer.add_scalar('test loss', test_loss / (batch_idx + 1), epoch * len(validation_loader)+ batch_idx) #!#
        writer.add_scalar('test accuracy', 100. * correct / total, epoch * len(validation_loader)+ batch_idx) #!#

    print('# TEST : Loss: ({:.4f}) | Acc: ({:.2f}%) ({}/{})'
          .format(test_loss / (batch_idx + 1), 100. * correct / total, correct, total))

        

In [11]:
def save_checkpoint(directory, state, filename='latest_1.tar.gz'):

    if not os.path.exists(directory):
        os.makedirs(directory)

    model_filename = os.path.join(directory, filename)
    torch.save(state, model_filename)
    print("=> saving checkpoint")

def load_checkpoint(directory, filename='latest_1.tar.gz'):

    model_filename = os.path.join(directory, filename)
    if os.path.exists(model_filename):
        print("=> loading checkpoint")
        state = torch.load(model_filename)
        return state
    else:
        return None

In [12]:
start_epoch = 0

checkpoint = load_checkpoint(default_directory, filename='vgg19_prelu_dropblock_cosine_1.tar.gz')

if not checkpoint:
    pass
else:
    start_epoch = checkpoint['epoch'] + 1
    model.load_state_dict(checkpoint['state_dict'])

for epoch in range(start_epoch, 200):

    train(epoch)
    
    save_checkpoint(default_directory, {
        'epoch': epoch,
        'model': model,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }, filename='vgg19_prelu_dropblock_cosine_1.tar.gz')
    test(epoch)  
    
now = time.gmtime(time.time() - start_time)
print('{} hours {} mins {} secs for training'.format(now.tm_hour, now.tm_min, now.tm_sec))

Epoch: 0 | Batch_idx: 0 |  Loss_1: (2.3088) | Acc_1: (6.25%) (8/128)
Epoch: 0 | Batch_idx: 10 |  Loss_1: (2.3040) | Acc_1: (10.16%) (143/1408)
Epoch: 0 | Batch_idx: 20 |  Loss_1: (2.2791) | Acc_1: (13.47%) (362/2688)
Epoch: 0 | Batch_idx: 30 |  Loss_1: (2.2272) | Acc_1: (15.50%) (615/3968)
Epoch: 0 | Batch_idx: 40 |  Loss_1: (2.1677) | Acc_1: (17.47%) (917/5248)
Epoch: 0 | Batch_idx: 50 |  Loss_1: (2.1165) | Acc_1: (19.13%) (1249/6528)
Epoch: 0 | Batch_idx: 60 |  Loss_1: (2.0682) | Acc_1: (20.68%) (1615/7808)
Epoch: 0 | Batch_idx: 70 |  Loss_1: (2.0284) | Acc_1: (22.06%) (2005/9088)
Epoch: 0 | Batch_idx: 80 |  Loss_1: (2.0012) | Acc_1: (22.98%) (2383/10368)
Epoch: 0 | Batch_idx: 90 |  Loss_1: (1.9756) | Acc_1: (24.03%) (2799/11648)
Epoch: 0 | Batch_idx: 100 |  Loss_1: (1.9497) | Acc_1: (25.24%) (3263/12928)
Epoch: 0 | Batch_idx: 110 |  Loss_1: (1.9303) | Acc_1: (26.11%) (3709/14208)
Epoch: 0 | Batch_idx: 120 |  Loss_1: (1.9135) | Acc_1: (26.87%) (4161/15488)
Epoch: 0 | Batch_idx: 130 |